In [108]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Import modules
1. Builder => for building a model.
2. pandas => for managing data.
3. os => for walking into directory to get files.
4. cleaner => for cleaning data.

In [109]:
from builder import Builder
import pandas as pd
import os
from cleaner import Cleaner

In [110]:
builder = Builder()
model = builder.build_model(type_of_model="LinearRegression")

In [111]:
"test"

'test'

In [112]:
for dirname, _, filenames in os.walk('/Covid-Regression-MortalityVSPopulation/datasets'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [113]:
#data_path = '/usr/src/Covid-Regression-MortalityVSPopulation/datasets/covid-vaccination-vs-death_ratio.csv'
data_path = 'C:\\Users\\Glaciiaz\\Documents\\GitHub\\MLOps-Regression-Machine-Learning-Problem\\Covid-Regression-MortalityVSPopulation\\datasets\\covid-vaccination-vs-death_ratio.csv'

In [114]:
df = pd.read_csv(data_path)

In [115]:
df.loc[df['country'] == 'Thailand']

,Unnamed: 0,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,New_deaths,population,ratio
21441,21441,Thailand,THA,2021-03-24,102050.0,96188.0,5862.0,0,70009982.0,0.137392
21442,21442,Thailand,THA,2021-03-30,180477.0,151413.0,29064.0,0,70009982.0,0.216273
21443,21443,Thailand,THA,2021-04-01,204642.0,167235.0,37407.0,0,70009982.0,0.238873
21444,21444,Thailand,THA,2021-04-02,231882.0,189666.0,42216.0,0,70009982.0,0.270913
21445,21445,Thailand,THA,2021-04-03,244254.0,201864.0,42390.0,1,70009982.0,0.288336
...,...,...,...,...,...,...,...,...,...,...
21653,21653,Thailand,THA,2021-11-15,85012905.0,45374194.0,36855732.0,45,70009982.0,64.811035
21654,21654,Thailand,THA,2021-11-16,85412126.0,45488767.0,37114546.0,62,70009982.0,64.974687
21655,21655,Thailand,THA,2021-11-17,86071507.0,45700036.0,37527295.0,56,70009982.0,65.276457
21656,21656,Thailand,THA,2021-11-18,86890826.0,46005152.0,37939622.0,55,70009982.0,65.712275


In [116]:
df.isna().sum()

Unnamed: 0                 0
country                    0
iso_code                   0
date                       0
total_vaccinations         0
people_vaccinated          0
people_fully_vaccinated    0
New_deaths                 0
population                 0
ratio                      0
dtype: int64

In [117]:
df.head()

,Unnamed: 0,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,New_deaths,population,ratio
0,0,Afghanistan,AFG,2021-05-11,504502.0,448878.0,55624.0,12,40194523.0,1.116764
1,1,Afghanistan,AFG,2021-05-20,547901.0,470341.0,77560.0,10,40194523.0,1.170162
2,2,Afghanistan,AFG,2021-05-24,573277.0,476367.0,96910.0,10,40194523.0,1.185154
3,3,Afghanistan,AFG,2021-05-26,590454.0,479372.0,111082.0,19,40194523.0,1.192630
4,4,Afghanistan,AFG,2021-05-27,593313.0,479574.0,113739.0,14,40194523.0,1.193133


In [118]:
data = df.copy()

In [119]:
data.loc[data['country'] == 'Thailand']

,Unnamed: 0,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,New_deaths,population,ratio
21441,21441,Thailand,THA,2021-03-24,102050.0,96188.0,5862.0,0,70009982.0,0.137392
21442,21442,Thailand,THA,2021-03-30,180477.0,151413.0,29064.0,0,70009982.0,0.216273
21443,21443,Thailand,THA,2021-04-01,204642.0,167235.0,37407.0,0,70009982.0,0.238873
21444,21444,Thailand,THA,2021-04-02,231882.0,189666.0,42216.0,0,70009982.0,0.270913
21445,21445,Thailand,THA,2021-04-03,244254.0,201864.0,42390.0,1,70009982.0,0.288336
...,...,...,...,...,...,...,...,...,...,...
21653,21653,Thailand,THA,2021-11-15,85012905.0,45374194.0,36855732.0,45,70009982.0,64.811035
21654,21654,Thailand,THA,2021-11-16,85412126.0,45488767.0,37114546.0,62,70009982.0,64.974687
21655,21655,Thailand,THA,2021-11-17,86071507.0,45700036.0,37527295.0,56,70009982.0,65.276457
21656,21656,Thailand,THA,2021-11-18,86890826.0,46005152.0,37939622.0,55,70009982.0,65.712275


In [120]:
data = data.drop(data.loc[data['country'] != 'Thailand'].index)

In [121]:
data

,Unnamed: 0,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,New_deaths,population,ratio
21441,21441,Thailand,THA,2021-03-24,102050.0,96188.0,5862.0,0,70009982.0,0.137392
21442,21442,Thailand,THA,2021-03-30,180477.0,151413.0,29064.0,0,70009982.0,0.216273
21443,21443,Thailand,THA,2021-04-01,204642.0,167235.0,37407.0,0,70009982.0,0.238873
21444,21444,Thailand,THA,2021-04-02,231882.0,189666.0,42216.0,0,70009982.0,0.270913
21445,21445,Thailand,THA,2021-04-03,244254.0,201864.0,42390.0,1,70009982.0,0.288336
...,...,...,...,...,...,...,...,...,...,...
21653,21653,Thailand,THA,2021-11-15,85012905.0,45374194.0,36855732.0,45,70009982.0,64.811035
21654,21654,Thailand,THA,2021-11-16,85412126.0,45488767.0,37114546.0,62,70009982.0,64.974687
21655,21655,Thailand,THA,2021-11-17,86071507.0,45700036.0,37527295.0,56,70009982.0,65.276457
21656,21656,Thailand,THA,2021-11-18,86890826.0,46005152.0,37939622.0,55,70009982.0,65.712275


In [122]:
data.drop(columns=['Unnamed: 0'], axis=0, inplace=True)

In [123]:
cleaner = Cleaner(data)

In [124]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 21441 to 21657
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   country                  217 non-null    object 
 1   iso_code                 217 non-null    object 
 2   date                     217 non-null    object 
 3   total_vaccinations       217 non-null    float64
 4   people_vaccinated        217 non-null    float64
 5   people_fully_vaccinated  217 non-null    float64
 6   New_deaths               217 non-null    int64  
 7   population               217 non-null    float64
 8   ratio                    217 non-null    float64
dtypes: float64(5), int64(1), object(3)
memory usage: 17.0+ KB


In [125]:
new_data = cleaner.drop_columns(['iso_code'])

In [126]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 21441 to 21657
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   country                  217 non-null    object 
 1   date                     217 non-null    object 
 2   total_vaccinations       217 non-null    float64
 3   people_vaccinated        217 non-null    float64
 4   people_fully_vaccinated  217 non-null    float64
 5   New_deaths               217 non-null    int64  
 6   population               217 non-null    float64
 7   ratio                    217 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 15.3+ KB


In [127]:
new_data['country'].loc[new_data['population'] == new_data['population'].max()].head()

21441    Thailand
21442    Thailand
21443    Thailand
21444    Thailand
21445    Thailand
Name: country, dtype: object

In [128]:
def add_ratio_data(df : pd.DataFrame):
    df['ratio_year'] = df.apply(lambda row: str(row.date)[0:4] + str(row.ratio), axis = 1)
    df['ratio_month'] = df.apply(lambda row: str(row.date)[5:7] + str(row.ratio), axis = 1)
    df['ratio_day'] = df.apply(lambda row: str(row.date)[8:10] + str(row.ratio), axis = 1)
    df['ratio_date'] = df.apply(lambda row: str(row.date) + str(row.ratio), axis = 1)
    return df

In [129]:
new_data = add_ratio_data(new_data)
new_data

,country,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,New_deaths,population,ratio,ratio_year,ratio_month,ratio_day,ratio_date
21441,Thailand,2021-03-24,102050.0,96188.0,5862.0,0,70009982.0,0.137392,20210.1373918364955443,030.1373918364955443,240.1373918364955443,2021-03-240.1373918364955443
21442,Thailand,2021-03-30,180477.0,151413.0,29064.0,0,70009982.0,0.216273,20210.2162734451210115,030.2162734451210115,300.2162734451210115,2021-03-300.2162734451210115
21443,Thailand,2021-04-01,204642.0,167235.0,37407.0,0,70009982.0,0.238873,20210.2388730795559981,040.2388730795559981,010.2388730795559981,2021-04-010.2388730795559981
21444,Thailand,2021-04-02,231882.0,189666.0,42216.0,0,70009982.0,0.270913,20210.270912796406661,040.270912796406661,020.270912796406661,2021-04-020.270912796406661
21445,Thailand,2021-04-03,244254.0,201864.0,42390.0,1,70009982.0,0.288336,20210.2883360261398153,040.2883360261398153,030.2883360261398153,2021-04-030.2883360261398153
...,...,...,...,...,...,...,...,...,...,...,...,...
21653,Thailand,2021-11-15,85012905.0,45374194.0,36855732.0,45,70009982.0,64.811035,202164.81103508925341,1164.81103508925341,1564.81103508925341,2021-11-1564.81103508925341
21654,Thailand,2021-11-16,85412126.0,45488767.0,37114546.0,62,70009982.0,64.974687,202164.97468746671011,1164.97468746671011,1664.97468746671011,2021-11-1664.97468746671011
21655,Thailand,2021-11-17,86071507.0,45700036.0,37527295.0,56,70009982.0,65.276457,202165.27645729147595,1165.27645729147595,1765.27645729147595,2021-11-1765.27645729147595
21656,Thailand,2021-11-18,86890826.0,46005152.0,37939622.0,55,70009982.0,65.712275,202165.71227514385019,1165.71227514385019,1865.71227514385019,2021-11-1865.71227514385019
